In [1]:
# NOTE: this is a custom cell that contains the common imports I personally 
# use these may/may not be necessary for the following examples

# DL framework
import tensorflow as tf

from datetime import datetime

# common packages
import numpy as np
import os # handling file i/o
import sys
import math
import time # timing epochs
import random

# for ordered dict when building layer components
import collections

# plotting pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib import colors # making colors consistent
from mpl_toolkits.axes_grid1 import make_axes_locatable # colorbar helper


# from imageio import imread # read image from disk
# + data augmentation
from scipy import ndimage
from scipy import misc


import pickle # manually saving best params
from sklearn.utils import shuffle # shuffling data batches
from tqdm import tqdm # display training progress bar

# const
SEED = 42

# Helper to make the output consistent
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# helper to create dirs if they don't already exist
def maybe_create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print("{} created".format(dir_path))
    else:
        print("{} already exists".format(dir_path))
    
def make_standard_dirs(saver=True, best_params=True, tf_logs=True):
    # `saver/` will hold tf saver files
    maybe_create_dir("saver")
    # `best_params/` will hold a serialized version of the best params
    # I like to keep this as a backup in case I run into issues with
    # the saver files
    maybe_create_dir("best_params")
    # `tf_logs/` will hold the logs that will be visable in tensorboard
    maybe_create_dir("tf_logs")

    
# set tf log level to supress messages, unless an error
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Important Version information
print("Python: {}".format(sys.version_info[:]))
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
reset_graph()

Python: (3, 5, 4, 'final', 0)
TensorFlow: 1.4.0
Default GPU Device: /device:GPU:0


In [2]:
make_standard_dirs()

saver already exists
best_params already exists
tf_logs already exists


In [3]:
FINAL_DIR = "../../dataset/record_holder/lesion/balanced/"
for _, _, files in os.walk(FINAL_DIR):
    files = sorted(files)
    for filename in files:
        print(filename)

test.tfrecords
train.tfrecords
val.tfrecords


In [4]:
# these two functions (get_model_params and restore_model_params) are 
# ad[a|o]pted from; 
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
def get_model_params():
    global_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {global_vars.op.name: value for global_vars, value in 
            zip(global_vars, tf.get_default_session().run(global_vars))}

def restore_model_params(model_params, g, sess):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: g.get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    sess.run(assign_ops, feed_dict=feed_dict)

# these two functions are used to manually save the best
# model params to disk
def save_obj(obj, name):
    with open('./best_params/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('./best_params/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Dataset

In [5]:
GLOBAL_SET_TYPE = None

def _parse_function(example_proto):
    global GLOBAL_SET_TYPE
    labelName = str(GLOBAL_SET_TYPE) + '/label'
    featureName = str(GLOBAL_SET_TYPE) + '/image'
    feature = {featureName: tf.FixedLenFeature([], tf.string),
               labelName: tf.FixedLenFeature([], tf.int64)}
    
    # decode
    parsed_features = tf.parse_single_example(example_proto, features=feature)
    
    # convert image data from string to number
    image = tf.decode_raw(parsed_features[featureName], tf.float32)
    image = tf.reshape(image, [224, 224, 3])
    
    label = tf.cast(parsed_features[labelName], tf.int64)
    
    image /= 255
    image -= 0.5
    image *= 2
    
    return image, label

In [6]:
def return_batched_iter(setType, data_params, sess):
    global GLOBAL_SET_TYPE
    global FINAL_DIR
    GLOBAL_SET_TYPE = setType
    
    filenames_ph = tf.placeholder(tf.string, shape=[None])

    dataset = tf.data.TFRecordDataset(filenames_ph)
    dataset = dataset.map(_parse_function)  # Parse the record into tensors.
    if GLOBAL_SET_TYPE != 'test':
        dataset = dataset.shuffle(buffer_size=data_params['buffer_size'])
    #dataset = dataset.shuffle(buffer_size=1)
    dataset = dataset.batch(data_params['batch_size'])
    dataset = dataset.repeat(1)
    
    iterator = dataset.make_initializable_iterator()
    
    tfrecords_file_name = str(GLOBAL_SET_TYPE) + '.tfrecords'
    tfrecord_file_path = os.path.join(FINAL_DIR, tfrecords_file_name)
    
    # initialize
    sess.run(iterator.initializer, feed_dict={filenames_ph: [tfrecord_file_path]})
    
    return iterator

In [7]:
def create_hyper_params():
    data_params = {}
    data_params['tf_logs'] = "zero"
    data_params['n_epochs'] = 60
    data_params['batch_size'] = 32
    data_params['buffer_size'] = 128 # for shuffling
    data_params['n_outputs'] = 1
    data_params['num_classes'] = 2
    
    data_params['path_to_GAN_ckpt'] = "./saved_gan_params/GAN_Model_3.ckpt"
    
    data_params['name_str'] = "1234"
    data_params["LOADFREEZE_STR"] = "discriminator/Gconv_["+data_params["name_str"]+"]"
    data_params["BEST_PARAMS_PATH"] = "./best_params/" + data_params["name_str"]

    data_params["init_lr"] = 1e-6

    return data_params

In [8]:
# # this function is adapted from https://github.com/ageron/handson-ml/blob/master/13_convolutional_neural_networks.ipynb
def augment_image(image, lrf_p=0.0, upf_p=0.0, r_degree=0.0, max_zoom=0.0, roll_num=0):
        
    n_image = image
    
    # Flip the image horizontally with upf_p% probability:
    if np.random.rand() < lrf_p:
        n_image = np.fliplr(n_image)

    # Flip the image vertically with lrf_p% probability:
    if np.random.rand() < upf_p:
        n_image = np.flipud(n_image)

    # 50% chance roll image horiz
    if np.random.rand() < 0.5:
        val = int(np.random.rand() * roll_num)
        # 50% left or right
        if np.random.rand() < 0.5:
            n_image = np.roll(n_image, val, axis=1)
        else:
            n_image = np.roll(n_image, -val, axis=1)
        
    # 50% chance roll image vert
    if np.random.rand() < 0.5:
        val = int(np.random.rand() * roll_num)
        # 50% left or right
        if np.random.rand() < 0.5:
            n_image = np.roll(n_image, val, axis=0)
        else:
            n_image = np.roll(n_image, -val, axis=0)

    
#     if r_degree > 0.0:
#         r_degree *= np.random.rand()
#         # randomly choose rotation direction
#         r_dir = (1 if np.random.rand() > 0.5 else -1)
#         n_image = ndimage.interpolation.rotate(n_image, r_dir*r_degree, reshape=False)
    

    return n_image

In [9]:
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
he_init = tf.contrib.layers.variance_scaling_initializer()

In [10]:
def build_graph(data_params):
    
    reset_graph()
    g = tf.Graph()
    with g.as_default():
        IMG_HEIGHT, IMG_WIDTH, CHANNELS = 224,224,3

        with tf.name_scope("inputs"):
            X = tf.placeholder(tf.float32, shape=[None, IMG_HEIGHT, IMG_WIDTH, CHANNELS], name="X")
            y_raw = tf.placeholder(tf.int64, shape=[None, data_params["n_outputs"]], name="y_input")
            y = tf.cast(y_raw, tf.float32, name="label")

            # for training/evaluating
            training = tf.placeholder_with_default(False, shape=[], name="training")

        
        with tf.name_scope("gan"):
            # 224x224
            conv1_1 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=3,
                                       strides=2, padding="SAME",
                                       activation=tf.nn.elu, 
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       kernel_regularizer=regularizer,
                                       bias_initializer=tf.zeros_initializer(), trainable=True,
                                       name="discriminator/Gconv_1")

            # 112x112
            conv1_2 = tf.layers.conv2d(conv1_1, 128, kernel_size=3, strides=2, padding='same', 
                                       activation=tf.nn.elu,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       kernel_regularizer=regularizer,
                                       bias_initializer=tf.zeros_initializer(), trainable=True,
                                       name="discriminator/Gconv_2")

            # 56x56
            conv2_1 = tf.layers.conv2d(conv1_2, 256, kernel_size=3, strides=2, padding='same', 
                                       activation=tf.nn.elu,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       kernel_regularizer=regularizer,
                                       bias_initializer=tf.zeros_initializer(), trainable=True,
                                       name="discriminator/Gconv_3")
            
#             

            # 28x28x256
            conv2_2 = tf.layers.conv2d(conv2_1, 512, kernel_size=3, strides=2, padding='same', 
                                       activation=tf.nn.elu,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       kernel_regularizer=regularizer,
                                       bias_initializer=tf.zeros_initializer(), trainable=True,
                                       name="discriminator/Gconv_4")
#             
            # 14x14x512
            conv2_3 = tf.layers.conv2d(conv2_2, 768, kernel_size=3, strides=1, padding='same', 
                                       activation=tf.nn.elu,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       kernel_regularizer=regularizer,
                                       bias_initializer=tf.zeros_initializer(), trainable=True,
                                       name="discriminator/my_conv")
            conv2_2 = tf.layers.max_pooling2d(inputs=conv2_3, pool_size=[2,2], strides=2, name="max_pool_myc")
            #7x7x768
        
            # reshape
            last_shape = int(np.prod(conv2_2.get_shape()[1:]))
            print("last_shape: ", last_shape)
            pool_flat = tf.reshape(conv2_2, shape=[-1, last_shape])
            pool_flat_drop = tf.layers.dropout(pool_flat, 0.5, training=training, name="entry_drop")

            fc1 = tf.layers.dense(pool_flat_drop, 256, activation=tf.nn.elu, name="fc1")
            fc1_drop = tf.layers.dropout(fc1, 0.5, training=training, name="fc1_drop")
            
            fc2 = tf.layers.dense(fc1_drop, 64, activation=tf.nn.elu, name="fc2")
            
            logits = tf.layers.dense(fc2, data_params["n_outputs"], name="logits")
            preds = tf.sigmoid(logits, name="preds")
            
        with tf.name_scope("loss"):
            xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
            batch_loss = tf.reduce_mean(xentropy, name="loss")

        with tf.name_scope("train"):
            optimizer = tf.train.AdamOptimizer(learning_rate=data_params["init_lr"],
                                               beta1=0.9,
                                               beta2=0.999,
                                               epsilon=1e-08,
                                               use_locking=False,
                                               name='Adam')
            
            all_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            freeze_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=data_params["LOADFREEZE_STR"])
            train_vars = [var for var in all_vars if var not in freeze_vars]
            print("num vars:", len(all_vars), "= frozen(", len(freeze_vars), ")", "+ train(", len(train_vars), ")")

            training_op = optimizer.minimize(batch_loss, var_list=train_vars, name="training_op")

        with tf.name_scope("save_session"):
            init_global = tf.global_variables_initializer()
            init_local = tf.local_variables_initializer()
            saver = tf.train.Saver()

        # Ops: training metrics
        with tf.name_scope("metrics"):
            # ================================== performance
            with tf.name_scope("common"):
                #preds = tf.nn.softmax(logits, name="prediction")
                #y_true_cls = tf.argmax(y,1)
                #y_pred_cls = tf.argmax(preds,1)
                y_true_cls = tf.greater_equal(y, 0.5)
                y_pred_cls = tf.greater_equal(preds, 0.5)

                correct_prediction = tf.equal(y_pred_cls, y_true_cls, name="correct_predictions")
                batch_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            with tf.name_scope("train_metrics") as scope:
                train_auc, train_auc_update = tf.metrics.auc(labels=y, predictions=preds)
                train_acc, train_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                train_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                train_met_reset_op = tf.variables_initializer(train_acc_vars, name="train_met_reset_op")
            with tf.name_scope("val_metrics") as scope:
                val_auc, val_auc_update = tf.metrics.auc(labels=y, predictions=preds)
                val_acc, val_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                val_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                val_met_reset_op = tf.variables_initializer(val_acc_vars, name="val_met_reset_op")
            with tf.name_scope("test_metrics") as scope:
                test_auc, test_auc_update = tf.metrics.auc(labels=y, predictions=preds)
                test_acc, test_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                test_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                test_acc_reset_op = tf.variables_initializer(test_acc_vars, name="test_met_reset_op")

            # =============================================== loss 
            with tf.name_scope("train_loss_eval") as scope:
                train_mean_loss, train_mean_loss_update = tf.metrics.mean(batch_loss)
                train_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                train_loss_reset_op = tf.variables_initializer(train_loss_vars, name="train_loss_reset_op")
            with tf.name_scope("val_loss_eval") as scope:
                val_mean_loss, val_mean_loss_update = tf.metrics.mean(batch_loss)
                val_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                val_loss_reset_op = tf.variables_initializer(val_loss_vars, name="val_loss_reset_op")
            with tf.name_scope("test_loss_eval")as scope:
                test_mean_loss, test_mean_loss_update = tf.metrics.mean(batch_loss)
                test_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                test_loss_reset_op = tf.variables_initializer(test_loss_vars, name="test_loss_rest_op")

        # --- create collections
        for node in (saver, init_global, init_local):
            g.add_to_collection("save_init", node)
        for node in (X, y_raw, training, training_op):
            g.add_to_collection("main_ops", node)
        for node in (preds, y_true_cls, y_pred_cls, correct_prediction):
            g.add_to_collection("preds", node)
        for node in (train_auc, train_auc_update, train_acc, train_acc_update, train_met_reset_op):
            g.add_to_collection("train_metrics", node)
        for node in (val_auc, val_auc_update, val_acc, val_acc_update, val_met_reset_op):
            g.add_to_collection("val_metrics", node)
        for node in (test_auc, test_auc_update, test_acc, test_acc_update, test_acc_reset_op):
            g.add_to_collection("test_metrics", node)
        for node in (train_mean_loss, train_mean_loss_update, train_loss_reset_op):
            g.add_to_collection("train_loss", node)
        for node in (val_mean_loss, val_mean_loss_update, val_loss_reset_op):
            g.add_to_collection("val_loss", node)
        for node in (test_mean_loss, test_mean_loss_update, test_loss_reset_op):
            g.add_to_collection("test_loss", node)
        g.add_to_collection("logits", logits)
            
        # ===================================== tensorboard
        with tf.name_scope("tensorboard_writer") as scope:
            epoch_train_loss_scalar = tf.summary.scalar('train_epoch_loss', train_mean_loss)
            epoch_train_acc_scalar = tf.summary.scalar('train_epoch_acc', train_acc)
            epoch_train_auc_scalar = tf.summary.scalar('train_epoch_auc', train_auc)
            epoch_train_write_op = tf.summary.merge([epoch_train_loss_scalar, epoch_train_acc_scalar, epoch_train_auc_scalar], name="epoch_train_write_op")

            # ===== epoch, validation
            epoch_validation_loss_scalar = tf.summary.scalar('validation_epoch_loss', val_mean_loss)
            epoch_validation_acc_scalar = tf.summary.scalar('validation_epoch_acc', val_acc)
            epoch_validation_auc_scalar = tf.summary.scalar('validation_epoch_auc', val_auc)
            epoch_validation_write_op = tf.summary.merge([epoch_validation_loss_scalar, epoch_validation_acc_scalar, epoch_validation_auc_scalar], name="epoch_validation_write_op")
        
        for node in (epoch_train_write_op, epoch_validation_write_op):
            g.add_to_collection("tensorboard", node)

    return g

In [11]:
def train_graph(g, data_params):
    
    #####################################
    # ----------- augmentation
    ##### training
    AUG_BOOL = True
    LRF_P = 0.5
    UPF_p = 0.5
    ROLL_NUM = 25
    
    ##### validation
    VAL_AUG_BOOL = True
    VAL_LRF_P = 0.3
    VAL_UPF_P = 0.3
    ######################################
    
    saver, init_global, init_local = g.get_collection("save_init")
    X, y_raw, training, training_op = g.get_collection("main_ops")
    preds, y_true_cls, y_pred_cls, _ = g.get_collection("preds")
    train_auc, train_auc_update, train_acc, train_acc_update, train_met_reset_op = g.get_collection("train_metrics")
    val_auc, val_auc_update, val_acc, val_acc_update, val_met_reset_op = g.get_collection("val_metrics")
    train_mean_loss, train_mean_loss_update, train_loss_reset_op = g.get_collection("train_loss")
    val_mean_loss, val_mean_loss_update, val_loss_reset_op = g.get_collection("val_loss")
    epoch_train_write_op, epoch_validation_write_op = g.get_collection("tensorboard")
#     next_tr_element, next_val_element, _ = g.get_collection("data_sets")

    train_writer = tf.summary.FileWriter(os.path.join("tf_logs",data_params['tf_logs'],
                                                      "train", data_params['name_str']))
    val_writer = tf.summary.FileWriter(os.path.join("tf_logs",data_params['tf_logs'],
                                                    "validation", data_params['name_str']))
    
    best_val_loss = np.inf
    
    with tf.Session(graph=g) as sess:
        
        sess.run([init_global, init_local])
        
        #####################################################################
        ## Load GAN params
        const_lookup = 0
        max_lookup = 5
        conv1_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                               scope="discriminator/Gconv_1")
        conv1_init = sess.run(conv1_vars)
        print(conv1_init[const_lookup][const_lookup][const_lookup][const_lookup][:max_lookup])
        
        # Load vars according to "LOADFREEZE_STR"
        reuse_vars = g.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                           scope=data_params["LOADFREEZE_STR"]) 
        reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
        if data_params['name_str'] != "0":
            restore_saver = tf.train.Saver(reuse_vars_dict)
            restore_saver.restore(sess, data_params['path_to_GAN_ckpt'])
        conv1_restore = sess.run(conv1_vars)
        
        # ensure vars are loaded correctly
        print("loaded {} vars".format(len(reuse_vars_dict)))
        print(conv1_restore[const_lookup][const_lookup][const_lookup][const_lookup][:max_lookup])
        #########################################################################

        for e in tqdm(range(1,data_params['n_epochs']+1)):
            sess.run([val_met_reset_op,val_loss_reset_op,train_met_reset_op,train_loss_reset_op])
            # training
            tr_iter = return_batched_iter('train', data_params, sess)
            next_tr_element = tr_iter.get_next()
            
            # loop entire training set
            while True:
                try:
                    data, target = sess.run(next_tr_element)
                    data_a = [augment_image(np.copy(img), lrf_p=LRF_P, 
                                            upf_p=UPF_p, roll_num=ROLL_NUM) for img in data]
                    target = np.reshape(target, (target.shape[0], 1))
                    sess.run([training_op, train_auc_update, train_acc_update, train_mean_loss_update], 
                             feed_dict={X:data_a, y_raw:target, training:True})
                except tf.errors.OutOfRangeError:
                    break
        
            # write average for epoch
            summary = sess.run(epoch_train_write_op)    
            train_writer.add_summary(summary, e)
            train_writer.flush()

            # run validation
            for i in range(3):
                val_iter = return_batched_iter('val', data_params, sess)
                next_val_element = val_iter.get_next()
                while True:
                    try:
                        Xb, yb = sess.run(next_val_element)
                        Xb_a = [augment_image(np.copy(img), lrf_p=VAL_LRF_P, 
                                              upf_p=VAL_UPF_P) for img in Xb]
                        yb = np.reshape(yb, (yb.shape[0], 1))
                        sess.run([val_auc_update, val_acc_update, val_mean_loss_update], 
                                 feed_dict={X:Xb_a, y_raw:yb})
                    except tf.errors.OutOfRangeError:
                        break

            # check for (and save) best validation params here
            cur_loss, cur_acc = sess.run([val_mean_loss, val_acc])
            if cur_loss < best_val_loss:
                best_val_loss = cur_loss
                best_params = get_model_params()
                save_obj(best_params, data_params['name_str'])
                print("bst prms svd: val acc: {:.3f}% loss: {:.4f}".format(cur_acc*100, cur_loss))

            summary = sess.run(epoch_validation_write_op) 
            val_writer.add_summary(summary, e)
            val_writer.flush()
        
        train_writer.close()
        val_writer.close()
    return sess
print("ready")

ready


In [12]:
reset_graph()
data_params = create_hyper_params()
print(data_params)
print(data_params["LOADFREEZE_STR"])
g = build_graph(data_params)
sess = train_graph(g, data_params)

{'path_to_GAN_ckpt': './saved_gan_params/GAN_Model_3.ckpt', 'tf_logs': 'zero', 'batch_size': 32, 'LOADFREEZE_STR': 'discriminator/Gconv_[1234]', 'n_epochs': 60, 'buffer_size': 128, 'n_outputs': 1, 'BEST_PARAMS_PATH': './best_params/1234', 'name_str': '1234', 'num_classes': 2, 'init_lr': 1e-06}
discriminator/Gconv_[1234]
last_shape:  37632
num vars: 16 = frozen( 8 ) + train( 8 )
[-0.04330047 -0.03859741  0.00831781 -0.05688733 -0.00263712]
INFO:tensorflow:Restoring parameters from ./saved_gan_params/GAN_Model_3.ckpt


  0%|          | 0/60 [00:00<?, ?it/s]

loaded 8 vars
[-0.02986679  0.08771463 -0.07817581 -0.01173449  0.03679096]


  2%|▏         | 1/60 [00:06<06:15,  6.37s/it]

bst prms svd: val acc: 55.273% loss: 0.6868


...

 37%|███▋      | 22/60 [01:44<03:00,  4.74s/it]

bst prms svd: val acc: 56.121% loss: 0.6773


 47%|████▋     | 28/60 [02:13<02:33,  4.78s/it]

bst prms svd: val acc: 58.182% loss: 0.6772


100%|██████████| 60/60 [05:11<00:00,  5.19s/it]


In [13]:
#     # Tensorboard
#     now = datetime.now().strftime("%d%b%Y_%H%M%S")
#     root_logdir = "tf_logs/final_discrim_class/" + NAME_STR + "_3"
#     logdir = "{}/{}/".format(root_logdir, now)

## Eval

In [14]:
reset_graph()
print(data_params["LOADFREEZE_STR"])
data_params = create_hyper_params()
g2 = build_graph(data_params)
best_params = load_obj(data_params['name_str'])
with tf.Session(graph=g2) as sess:
    confusion_mat = tf.Variable( tf.zeros([data_params["num_classes"],data_params["num_classes"]], dtype=tf.int32 ), name='confusion')
    sess.run(tf.global_variables_initializer())
    saver, init_global, init_local = g2.get_collection("save_init")
    X, y_raw, training, training_op = g2.get_collection("main_ops")
    preds, y_true_cls, y_pred_cls, _ = g2.get_collection("preds")
    test_auc, test_auc_update, test_acc, test_acc_update, test_acc_reset_op = g2.get_collection("test_metrics")
    test_mean_loss, test_mean_loss_update, test_loss_reset_op = g2.get_collection("test_loss")
    
    restore_model_params(model_params=best_params, g=g2, sess=sess)
    sess.run([test_acc_reset_op, test_loss_reset_op])
    
    test_iter = return_batched_iter('test', data_params, sess)
    next_test_element = test_iter.get_next()
    while True:
        try:
            Xb, yb = sess.run(next_test_element)
            yb = np.reshape(yb, (yb.shape[0], 1))
            sess.run([test_auc_update, test_acc_update, test_mean_loss_update], feed_dict={X:Xb, y_raw:yb})
            jj = y_true_cls.eval(feed_dict={y_raw: yb})
            batch_conf_matrix = tf.confusion_matrix(labels = y_true_cls.eval(feed_dict={y_raw: yb}).reshape(-1),
                                                predictions = y_pred_cls.eval(feed_dict={X: Xb}).reshape(-1),
                                                num_classes=data_params["num_classes"])
            sess.run(confusion_mat.assign(confusion_mat.eval() + batch_conf_matrix.eval()))
        except tf.errors.OutOfRangeError:
            break    
    
    # print
    final_test_acc, final_test_loss, final_test_auc = sess.run([test_acc, test_mean_loss, test_auc])
    print("test auc: {:.3f}% acc: {:.3f}% loss: {:.5f}".format(final_test_auc*100, 
                                                              final_test_acc*100,
                                                              final_test_loss))
    print(confusion_mat.eval())

discriminator/Gconv_[1234]
last_shape:  37632
num vars: 16 = frozen( 8 ) + train( 8 )
test auc: 68.184% acc: 64.826% loss: 0.64596
[[111  61]
 [ 60 112]]


```
0    [[112  60]    test auc: 67.599% acc: 62.791% loss: 0.64508
      [ 68 104]]
      
1    [[113  59]    test auc: 67.957% acc: 64.535% loss: 0.64551
      [ 63 109]]

12   [[109  63]   test auc: 67.847% acc: 63.663% loss: 0.64502
      [ 62 110]]

123   [[104  68]   test auc: 67.430% acc: 62.500% loss: 0.64825
       [ 61 111]]
       
1234  [[111  61]   test auc: 68.184% acc: 64.826% loss: 0.64596
       [ 60 112]]

    
```

In [20]:
print("{:4} - sens {:.3f}\% prec: {:.3f}\%".format("0",(104/(104+68))*100, (104/(104+60))*100))
print("{:4} - sens {:.3f}\% prec: {:.3f}\%".format("1",(109/(109+63))*100, (109/(109+59))*100))
print("{:4} - sens {:.3f}\% prec: {:.3f}\%".format("12",(110/(110+62))*100, (110/(110+63))*100))
print("{:4} - sens {:.3f}\% prec: {:.3f}\%".format("123",(111/(111+61))*100, (111/(111+68))*100))
print("{:4} - sens {:.3f}\% prec: {:.3f}\%".format("1234",(112/(112+60))*100, (112/(112+61))*100))

0    - sens 60.465\% prec: 63.415\%
1    - sens 63.372\% prec: 64.881\%
12   - sens 63.953\% prec: 63.584\%
123  - sens 64.535\% prec: 62.011\%
1234 - sens 65.116\% prec: 64.740\%
